# Task 2 Generate Sparse Representations 

#### Student Name: Xiaofan Zhang
#### Student ID: 28338707

Date: 03/06/2018

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,: 
* re 
* os
* nltk
* nltk.tokenize

In [1]:
import nltk
import os
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
import re

In [2]:
# remove all stopwords
# remove words whose frequencies is larger than 132 less than 3
# sort the list
# add index number behind every tokens

## 1. Unigram vocabulary 
## All raw tokens

In [3]:
txt_file_path = "./txt_files"

In [4]:
tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)?") 
def main(file):
    
    file_object = open(file, "r")
    lines = file_object.readlines()
    
    unigram_tokens = []
    for line in lines:
        line = line.strip().lower()
        
        token_list = tokenizer.tokenize(line)
        for i in range(len(token_list)):
            unigram_tokens.append(token_list[i])
         
    file_object.close()
    # delete duplicate tokens
    return(set(unigram_tokens))


In [6]:
all_list = []
for xfile in os.listdir(txt_file_path):
    xfile = os.path.join(txt_file_path, xfile)
    if os.path.isfile(xfile) and xfile.endswith('.txt'): 
        all_list += main(xfile)
# len(all_list)
all_list

['fine',
 'nice',
 'stages',
 'about',
 'god',
 'is',
 'talking',
 'too',
 'sum',
 'wags',
 'unique',
 'suppose',
 'whistle',
 'blue',
 'gimmicky',
 'technologically',
 "who's",
 'harmless',
 'own',
 'right',
 'regular',
 'm_p_',
 'living',
 'next',
 'each',
 'discuss',
 "you'll",
 'terms',
 'scribble',
 "we've",
 'its',
 'information',
 'pitched',
 'chasing',
 'thing',
 'animal',
 'bas',
 'technology',
 'junky',
 'issue',
 'personally',
 'hit',
 'they',
 'uh-huh',
 'yeah',
 'piece',
 'greece',
 'did',
 'draw',
 'start',
 'supposed',
 'sold',
 'forever',
 'reason',
 'health',
 'control',
 'depends',
 'oh',
 'even',
 'though',
 'video',
 'chases',
 'retailer',
 'th',
 'view',
 'certain',
 'pe',
 'again',
 'sudden',
 'doing',
 'country',
 'standard',
 'premium',
 'being',
 'dunno',
 'combine',
 'animals',
 'come',
 'looks',
 "i'd",
 'really',
 'telephones',
 'down',
 'personality',
 'aware',
 'yep',
 'received',
 'space',
 'slipped',
 'get',
 'eighteen',
 'f',
 'will',
 'natural',
 'crai

## Get the stop words lists

In [5]:
# get the stop words lists
stop_file = open("stopwords_en.txt", "r")
stop_list = stop_file.read().split('\n')
# stop_file.close() at the end of code

## Remove all stop words


In [6]:
# remove all stop words
stop_set = set(stop_list)
stopped_tokens = [ww for ww in all_list if ww not in stop_set]

## Remove all words whose length less than 3

In [7]:
# remove all words whose length less than 3
new_tokens = [w for w in stopped_tokens if len(w) >2]
# len(new_tokens)

## Remove words whose frequencies is larger than 132 


In [8]:
# remove words whose frequencies is larger than 132 
wordfreq = []
fdist = FreqDist(all_list)   
for v in new_tokens:
    if fdist[v] <= 132:
        wordfreq.append(v)
# sorted(fdist.items())

In [9]:
wordfreq = sorted(set(wordfreq))

## Final vocab list

In [10]:
final_list = ['{0}:{1}'.format(i, wordfreq.index(i)) for i in wordfreq]

## Write file

In [12]:
# write file
vocab = open("vocab.txt","w")
vocab_str = "\n".join(final_list)
vocab.write(vocab_str)
vocab.close()

In [13]:
# create a dictionary for store vocab and index
vocab_dic = dict()
for ii in wordfreq:
    vocab_dic[ii] = wordfreq.index(ii)

## 2. Topic_seg

Assumption: I assumed that "after being preprocessed" means preprocessed in task1

In [14]:
def seg(txt):
    txt_file = open(txt, "r")
    l = []# list for storing boolean number
    for line in txt_file.readlines():
        if line[0] == ' ':# start with a space append "0"
            l.append(0)
        if line[0] == '*': # start with "*" append "1" at the previous line
            l[-1] = 1
    return l

In [15]:
_topic = ""
for xfile in os.listdir(txt_file_path):
    if xfile.endswith('.txt'):
        # get file name
        x = re.findall(r'\w{2}\d{4}\w{1}|\w{2}\d{4}', xfile)[0]

    one_topic = ""
    xfile = os.path.join(txt_file_path, xfile)
    if os.path.isfile(xfile) and xfile.endswith('.txt'): 
        for se in seg(xfile):
            one_topic += str(se) + ","

        _topic += x +":"+ one_topic[0:-1] +'\n'
_topic = _topic[0:-1]

topic_segs = open("topic_segs.txt","w")
topic_segs.write(_topic)
topic_segs.close()

## 3. Sparse_files


In [16]:
# 'count' function for count frequency
def count(arr):
    count = dict()
    for w in arr:
        if w in count:
            count[w] += 1
        else:
            count[w] = 1
    return count

### Sparse function for generating corresponding sparse file
Format :$index:frequency$

In [17]:
def sparse(txt):
    txt_file = open(txt, "r")
    file_list = [] # whole file list
    for line in txt_file.readlines():
        file_token = []
        line = line.strip().lower()            
        token_list = tokenizer.tokenize(line)# each line in text file and store as a list

        for t in token_list:
            if t in vocab_dic:
                # corresponding index : frequency
                file_token.append(str(vocab_dic[t])+":"+ str(count(token_list)[t]))
        # delete empty list
        if file_token != []:
            tep_list = list(set(file_token))
            tep_list.sort(key=file_token.index)
            
            file_list.append(tep_list)

    return(file_list)
 

### Use sparse function to generate all files

In [18]:
# final output
for xfile in os.listdir(txt_file_path):
    if xfile.endswith('.txt'):
        x = xfile
    xfile = os.path.join(txt_file_path, xfile)
    if os.path.isfile(xfile) and xfile.endswith('.txt'): 
        seg_list = sparse(xfile)
        str2 = ""
        for item in seg_list:
            str1 = ""
            for i in item:
                str1 = str1+i + ","
                
            str1 = str1[0:-1]
            str2 += str1 +"\n"
        str2 = str2[0:-1]
        # write file
        thefile = open("sparse_files/"+x,"w")
        thefile.write(str2)
        thefile.close()


In [ ]:
stop_file.close()